# Introduction

In the 16-cycle experiment, three kinds of plots were used to compare cycles: paired histograms, density scatter plots, and four-histogram plots. Below we provide code for generating these plots for all possibly relevant comparisons in the data for convenience. For sake of speed, each dataset only includes 10000 randomly sampled points of the original dataset.

To begin, please run the next block of code by selecting it, and pressing "Shift+Enter" to load the data and prerequisite functions.

In [2]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from scipy.stats.stats import pearsonr as rho

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import pandas as pd
from math import log

df1 = pd.read_csv('T_sampleA.csv')
df2 = pd.read_csv('T_sampleB.csv')
df3 = pd.read_csv('M_sampleA.csv')
df4 = pd.read_csv('M_sampleB.csv')
df5 = pd.read_csv('N_sampleA.csv')
df6 = pd.read_csv('N_sampleB.csv')

text_file = open("labels.txt", "r")
labels = text_file.read().split('\t')
text_file.close()

alldata = [df1,df2,df3,df4,df5,df6]
datadict = {'TonsilA':0,'TonsilB':1,'MA':2,'MB':3,'NA':4,'NB':5}
cycldict = dict(zip(labels,range(0,64)))
channeldict = {'Hoechst':0,'PCNA':1,'Vimentin':2,'Tubulin':3}

def overlaparea(kde1,kde2,trapzpts):
    z1vals = kde1.evaluate(trapzpts)
    z2vals = kde2.evaluate(trapzpts)
    minvals = np.minimum(z1vals,z2vals)
    #plt.plot(trapzpts,minvals,label='Overlap')
    a = trapzpts[0]
    c = trapzpts[len(trapzpts)-1]
    dx = (c-a)/(len(trapzpts)-1)
    integ = (sum(minvals)-minvals[0]/2-minvals[len(trapzpts)-1]/2)*dx
    return(integ)

def histocompare(Marker_A,Marker_B,dataset1,dataset2):
    df1 = alldata[dataset1]
    df2 = alldata[dataset2]
    colA = df1.iloc[:,Marker_A];
    colB = df2.iloc[:,Marker_B];

    x = [log(y) for y in colA];
    y = [log(y) for y in colB];

    z1 = gaussian_kde(x)
    z2 = gaussian_kde(y)
    binchoice = np.linspace(min(x+y),max(x+y),100)
    
    plt.plot(binchoice,z1.evaluate(binchoice),label=labels[Marker_A])
    plt.plot(binchoice,z2.evaluate(binchoice),label=labels[Marker_B])
    plt.xlabel('Log-Intensity')
    plt.ylabel('Estimated PDF')
    plt.title('Overlap='+"{:.2f}".format(overlaparea(z1,z2,binchoice)))
    plt.legend()
    plt.show()

def dscatter(Marker_A,Marker_B,dataset):
    if Marker_A==Marker_B:
       print('Please Select Differing Markers (changes take a minute)')
        
    else:
        df = alldata[dataset]
        colA = df.iloc[:,Marker_A];
        colB = df.iloc[:,Marker_B];
    
        x = [log(y) for y in colA];
        y = [log(y) for y in colB];
        xy = np.vstack([x,y])
        z= gaussian_kde(xy)(xy)
    
        rhoval = rho(x,y)
        LineFit = np.polyfit(x, y, 1)

        
        plt.scatter(x,y,c=z,s=10,edgecolor='')
        plt.title('Correlation='+"{:.2f}".format(rhoval[0])+', Slope='+"{:.2f}".format(LineFit[0]))
        plt.xlabel(labels[Marker_A]+' Log-Intensity')
        plt.ylabel(labels[Marker_B]+' Log-Intensity')
        plt.show()

def fourcycle(dataset,channel):
    df = alldata[dataset]
    col3 = df.iloc[:,3+16*channel-1]
    col7 = df.iloc[:,7+16*channel-1];
    col12 = df.iloc[:,12+16*channel-1]
    col16 = df.iloc[:,16+16*channel-1];
    
    x3 = [log(y) for y in col3];
    x7 = [log(y) for y in col7];
    x12 = [log(y) for y in col12];
    x16 = [log(y) for y in col16];

    z3 = gaussian_kde(x3)
    z7 = gaussian_kde(x7)
    z12 = gaussian_kde(x12)
    z16 = gaussian_kde(x16)
    binchoice = np.linspace(min(x3+x7+x12+x16),max(x3+x7+x12+x16),100)
    
    plt.plot(binchoice,z3.evaluate(binchoice),label='Cycle3')
    plt.plot(binchoice,z7.evaluate(binchoice),label='Cycle7')
    plt.plot(binchoice,z12.evaluate(binchoice),label='Cycle12')
    plt.plot(binchoice,z16.evaluate(binchoice),label='Cycle16')
    plt.xlabel('Log-Intensity')
    plt.ylabel('Estimated PDF')
    plt.legend()
    plt.show()

You may run the code for any of the three types of plots you like below:

# Histogram Comparison of Two Channels

The code below allows you to choose one channel from one dataset, and another channel from another dataset. The overlapping area is also computed.

In [3]:
thing1 = interactive(histocompare,Marker_A=cycldict,Marker_B=cycldict,dataset1=datadict,dataset2=datadict)
display(thing1)


# Density Scatter Plot of Two Markers from Same Dataset

When the dataset is the same, we can make a scatter plot for any of the two channels. Here we also compute the Pearson correlation, denoted by "rho", and the slope of the line-of-best-fit. (The density calculation takes a little more time, so give a minute for changes to implement)

In [4]:
thing2 = interactive(dscatter,Marker_A=cycldict,Marker_B=cycldict,dataset=datadict)
display(thing2)

# Four Histogram Plot

For a handful of markers, we reused the antibodies at multiple different cycles in the same sample.

In [5]:
thing3 = interactive(fourcycle,dataset=datadict,channel=channeldict)
display(thing3)